# CSE 4334/5334 Programming Assignment 1 (P1)

## Spring 2016

## Due: 11:59pm Central Time, Thursday, March 3, 2016

In this assignment, you will implement a toy "search engine". More specifically, you code will read a corpus and produce TF-IDF vectors for documents in the corpus. Then, given a query string, you code will compute the cosine similarity between the query vector and the document vectors and return the document that gets the highest similarity score. 

The instructions on this assignment are written in an .ipynb file. You can use the following commands to install the iPython notebook viewer:

    pip install ipython

    pip install notebook (You might have to use sudo if you are installing them at system level)

To run the iPython notebook viewer, use the following command

    ipython notebook P1.ipynb

The above command will start a webservice at http://localhost:8888/ and display a '.ipynb' file.

### Requirements

* This assignment must be done individually. You must implement the whole assignment by yourself. Academic dishonety will have serious consequences.
* You are encouraged to discuss the assignment with other students, but you are not allowed to disclose your solution. 

### Dataset

We use a corpus of all the general election presidential debates from 1960 to 2012. We processed the corpus and provided you a .zip file, which includes 30 .txt files. Each of the 30 files contains the transcript of a debate and is named by the date of the debate. The .zip file can be downloaded from Blackboard ("Course Materials" > "Programming Assignment 1" > "Attached Files: presidential_debates.zip").

### Programming Language

1. You are required to use Python 3.5.1. You are required to submit a single .py file of your code. We will test your code under the particular version of Python 3.5.1. So make sure you develop your code using the same version. 

2. You are expected to use several modules in NLTK--a natural language processing toolkit for Python. NLTK doesn't come with Python by default. You need to install it and "import" it in your .py file. NLTK's website (http://www.nltk.org/index.html) provides a lot of useful information, including a book http://www.nltk.org/book/, as well as installation instructions (http://www.nltk.org/install.html).

3. In programming assignment 1, other than NLTK, you are not allowed to use any other non-standard Python package. However, you are free to use anything from the the Python Standard Library that comes with Python 3.5.1 (https://docs.python.org/3/library/).

### Tasks

You code should accomplish the following tasks:

(1) <b>Read</b> the 30 .txt files, each of which has the transcript of a presidential debate. The following code does it. Make sure to replace corpus_root by your directory where the files are stored.

Also, convert the text to lower case.

In [32]:
import os
corpus_root = './presidential_debates'
for filename in os.listdir(corpus_root):
    file = open(os.path.join(corpus_root, filename), "r", encoding='UTF-8')
    doc = file.read()
    file.close() 
    doc = doc.lower()

(2) <b>Tokenize</b> the content of each file. For this, you need a tokenizer. For example, the following piece of code uses a regular expression tokenizer to return all course numbers in a string. Play with it and edit it. You can change the regular expression and the string to observe different output results. 

For tokenizing the Presidential debate speeches, let's all use RegexpTokenizer(r'[a-zA-Z]+'). What tokens will it produce? What limitations does it have? 


In [33]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[A-Z]{2,3}[1-9][0-9]{3,3}')
tokens = tokenizer.tokenize("CSE4334 and CSE5334 are taught together. IE3013 is an undergraduate course.")
print(tokens)

['CSE4334', 'CSE5334', 'IE3013']


(3) Perform <b>stopword removal</b> on the obtained tokens. NLTK already comes with a stopword list, as a corpus in the "NLTK Data" (http://www.nltk.org/nltk_data/). You need to install this corpus. Follow the instructions at http://www.nltk.org/data.html. You can also find the instruction in this book: http://www.nltk.org/book/ch01.html (Section 1.2 Getting Started with NLTK). Basically, use the following statements in Python interpreter. A pop-up window will appear. Click "Corpora" and choose "stopwords" from the list.

In [34]:
import nltk
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

After the stopword list is downloaded, you will find a file "english" in folder nltk_data/corpora/stopwords, where folder nltk_data is the download directory in the step above. The file contains 127 stopwords. nltk.corpus.stopwords will give you this list of stopwords. Try the following piece of code.

In [35]:
from nltk.corpus import stopwords
print(stopwords.words('english'))
print(sorted(stopwords.words('english')))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

(4) Also perform <b>stemming</b> on the obtained tokens. NLTK comes with a Porter stemmer. Try the following code and learn how to use the stemmer.

In [36]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
stemmer.stem('studying')

'studi'

(5) Using the tokens, compute the <b>TF-IDF vector</b> for each document. Use the following equation that we learned in the lectures to calculate the term weights:  $$(1+log_{10}{tf_{t,d}})\times(log_{10}{\frac{N}{df_t}}).$$ Note that the TF-IDF vectors should be normalized (i.e., their lengths should be 1). 

Represent a TF-IDF vector by a dictionary. The following is a sample TF-IDF vector.

In [37]:
{'sanction': 0.014972337775895645, 'lack': 0.008576372825970286, 'regret': 0.009491784747267843, 'winter': 0.030424375278541155}

{'lack': 0.008576372825970286,
 'regret': 0.009491784747267843,
 'sanction': 0.014972337775895645,
 'winter': 0.030424375278541155}

(6) Given a query string, calculate the query vector. Compute the <b>cosine similarity</b> between the query vector and every document vector. Return the document that attains the highest cosine similarity score.

In calculating the query vector, don't consider IDF. I.e., use the following equation to calculate the term weights in the query vector: $$(1+log_{10}{tf_{t,d}}).$$
The vector should still be normalized. 

### What to Submit 

Submit through Blackboard your source code in a single .py file. You can use any standard Python library. The only non-standard library/package allowed for this assignment is NLTK.

The file must define at least the following functions:

* query(qstring): return the document that has the highest similarity score with respect to 'qstring'.

* getcount(token): return the total number of occurrences of a token in all documents.

* getidf(token): return the inverse document frequency of a token. If the token doesn't exist in the corpus, return 0. 

* docdocsim(filename1,filename2): return the cosine similarity betwen two speeches (files).

* querydocsim(qstring,filename): return the cosine similairty between a query string and a document. 

Some sample results that we should expect from a correct implementation:

###### Output set1:

* print(query("health insurance wall street"))

2012-10-03.txt

* print(getcount("health"))

483

* print("%.12f" % getidf("health"))

0.079181246048

* print("%.12f" % docdocsim("1960-09-26.txt", "1980-09-21.txt"))

0.068531196532

* print("%.12f" % querydocsim("health insurance wall street", "1996-10-06.txt"))

0.022032578028




###### Output set2:

* print(query("security conference ambassador"))

1960-10-21.txt

* print(getcount('attack'))

109

* print("%.12f" % getidf("agenda"))

0.363177902413

* print("%.12f" % docdocsim("1960-10-21.txt", "1980-09-21.txt"))

0.060203573673

* print("%.12f" % querydocsim("particular constitutional amendment", "2000-10-03.txt"))

0.012542376718




###### Output set3:

* print(query("particular constitutional amendment"))

1976-10-22.txt

* print(getcount('amend'))

90

* print("%.12f" % getidf("particular"))

0.176091259056

* print("%.12f" % docdocsim("1960-09-26.txt", "1960-10-21.txt"))

0.178876247414

* print("%.12f" % querydocsim("health insurance wall street", "2000-10-03.txt"))

0.007815215703

### Evaluation

We will evaluate your code by calling the functions specificed above. So, make sure to use the same function names, parameter names/types/orders as specified above. 